In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
#@ Loading the required libraries
import tensorflow as tf
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [3]:
#@ Loading the required dataset
data_train = pd.read_csv('/kaggle/input/fake-news/train.csv')
data_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
#@ Checking shape of dataset
data_train.shape

(20800, 5)

In [5]:
#@ Checking if there is any missing values
data_train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
#@ Let's drop the null values
data = data_train.dropna()

In [7]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
#@ Let's get Independent and Dependent features
X = data.drop('label', axis=1)
y = data['label']

In [9]:
#@ Checking shape after extracting features
print('The shape of X is:',X.shape)
print('The shape of y is:',y.shape)

The shape of X is: (18285, 4)
The shape of y is: (18285,)


In [10]:
#@ Defining vocabulary size
voc_size=5000

**One Hot Representation**

In [11]:
#@ Copying the X values as messages
messages = X.copy()
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [12]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [13]:
messages.reset_index(inplace=True)
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [14]:
#@ Let's apply the preprocessing in the dataset
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-z0-9]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 '15 civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic [photos]',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay 2 5 million releas everyon arrest dakota access pipelin',
 'fbi c

In [15]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [16]:
data['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [17]:
#@ Now, let's apply OHE in the corpus
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[4692, 3435, 260, 4994, 530, 4899, 2368, 1312, 1278, 1394],
 [3133, 509, 360, 847, 2595, 3170, 2830],
 [4248, 3123, 4203, 4198],
 [2235, 3505, 383, 3744, 3074, 1357, 437],
 [1903, 2595, 4572, 1696, 2154, 897, 2595, 4262, 1281, 88],
 [4274,
  4469,
  1579,
  4853,
  4715,
  2223,
  2626,
  2793,
  3952,
  2258,
  1618,
  2925,
  2778,
  2405,
  2830],
 [3196, 3526, 2111, 253, 3535, 447, 4331, 4612, 3827, 4030, 4766],
 [213, 1577, 1567, 831, 768, 1863, 2223, 4753, 3827, 4030, 4766],
 [2026, 2110, 4523, 4806, 1850, 2642, 4689, 3577, 2223, 660],
 [2056, 3748, 4049, 1998, 3689, 3846, 3851, 4086],
 [3435, 321, 3610, 2804, 3181, 4214, 698, 1680, 4751, 884, 33],
 [3074, 651, 530, 2642, 2223, 768],
 [2269, 35, 1040, 17, 2257, 3988, 2219, 386, 3224],
 [1439, 3031, 2539, 2982, 2152, 1545, 106, 3827, 4030, 4766],
 [2218, 2134, 3573, 762, 4522, 3827, 4030, 4766],
 [4822, 4034, 184, 1128, 3188, 3113, 981, 1767, 1776, 3437, 475, 273],
 [265, 353, 509],
 [3634, 590, 4029, 3062, 2223, 3318, 3180, 2830

In [18]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [19]:
onehot_repr[0]

[4692, 3435, 260, 4994, 530, 4899, 2368, 1312, 1278, 1394]

**Embedding representation**

In [20]:
#@ Let's apply post padding in each sentences
sent_length=20
embedded_docs = pad_sequences(onehot_repr, padding='post', maxlen=sent_length)
embedded_docs

array([[4692, 3435,  260, ...,    0,    0,    0],
       [3133,  509,  360, ...,    0,    0,    0],
       [4248, 3123, 4203, ...,    0,    0,    0],
       ...,
       [ 961, 2996,  239, ...,    0,    0,    0],
       [1845,  768, 4686, ...,    0,    0,    0],
       [4473, 1695, 2695, ...,    0,    0,    0]], dtype=int32)

In [21]:
embedded_docs[0]

array([4692, 3435,  260, 4994,  530, 4899, 2368, 1312, 1278, 1394,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [22]:
#@ Now, let's create the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100,))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
len(embedded_docs), y.shape

(18285, (18285,))

In [24]:
X_final = np.array(embedded_docs)
y_final=np.array(y)

In [25]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [26]:
#@ Applying train/test/splt
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
#@ Let's finally train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 35ms/step - loss: 0.3214 - accuracy: 0.8473 - val_loss: 0.1956 - val_accuracy: 0.9157
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1386 - accuracy: 0.9478 - val_loss: 0.2050 - val_accuracy: 0.9175
Epoch 3/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0928 - accuracy: 0.9655 - val_loss: 0.2142 - val_accuracy: 0.9173
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0663 - accuracy: 0.9769 - val_loss: 0.2850 - val_accuracy: 0.9200
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0426 - accuracy: 0.9855 - val_loss: 0.2804 - val_accuracy: 0.9148
Epoch 6/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0309 - accuracy: 0.9902 - val_loss: 0.3229 - val_accuracy: 0.9100
Epoch 7/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0258 - accuracy: 0.9919 - val_loss: 0.3716 - val_accuracy: 0.9143

- The model seems to perform well on the training data, with the training accuracy improving from 0.84 in the first epoch to 0.9961 in the ninth epoch. However, the validation accuracy does not improve as much, hovering around 0.91 throughout training.
- This suggests that the model is overfitting to the training data and not generalizing well to new data. The increasing validation loss in later epochs also supports this hypothesis. The model may benefit from additional regularization techniques, such as dropout or weight decay, to prevent overfitting.

In [27]:
#@ Adding Dropout layer to the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100,))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
#@ Applying train/test/split
X_final = np.array(embedded_docs)
y_final=np.array(y)
#@ Applying train/test/splt
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
#@ Let's finally train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 39ms/step - loss: 0.3212 - accuracy: 0.8425 - val_loss: 0.2025 - val_accuracy: 0.9153
Epoch 2/10
192/192 [==============================] - 7s 35ms/step - loss: 0.1451 - accuracy: 0.9434 - val_loss: 0.1940 - val_accuracy: 0.9218
Epoch 3/10
192/192 [==============================] - 7s 34ms/step - loss: 0.1066 - accuracy: 0.9616 - val_loss: 0.2145 - val_accuracy: 0.9216
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0789 - accuracy: 0.9737 - val_loss: 0.2246 - val_accuracy: 0.9200
Epoch 5/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0548 - accuracy: 0.9818 - val_loss: 0.2479 - val_accuracy: 0.9185
Epoch 6/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0450 - accuracy: 0.9853 - val_loss: 0.2571 - val_accuracy: 0.9158
Epoch 7/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0374 - accuracy: 0.9869 - val_loss: 0.3138 - val_accuracy: 0.905

- The result after applying dropout technique shows a slightly better validation accuracy than the previous result. This means that dropout helped reduce overfitting and improve the generalization of the model.
- The training accuracy decreased slightly, which is expected because dropout randomly drops out some connections during training, making it harder for the model to fit the training data perfectly. However, this trade-off is necessary to achieve better performance on unseen data.

In [29]:
# Define early stopping criteria
earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')

# Adding Dropout layer to the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100,))
model.add(Dense(1, activation='sigmoid'))

In [30]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model with early stopping
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, callbacks=[earlystop])

Epoch 1/100
192/192 [==============================] - 9s 36ms/step - loss: 0.3194 - accuracy: 0.8459 - val_loss: 0.1974 - val_accuracy: 0.9181
Epoch 2/100
192/192 [==============================] - 6s 34ms/step - loss: 0.1525 - accuracy: 0.9382 - val_loss: 0.1975 - val_accuracy: 0.9158
Epoch 3/100
192/192 [==============================] - 7s 35ms/step - loss: 0.1123 - accuracy: 0.9580 - val_loss: 0.2239 - val_accuracy: 0.9186
Epoch 4/100
192/192 [==============================] - 7s 35ms/step - loss: 0.0894 - accuracy: 0.9661 - val_loss: 0.2548 - val_accuracy: 0.9157
Epoch 4: early stopping


- The use of callbacks has stopped training early on the 6th epoch since the validation loss did not improve in the last 3 epochs. This is a good sign that our model is not overfitting and has reached a good level of accuracy. The final validation accuracy achieved by the model is 91.10%.

In [31]:
#@ Performance Metrics and Accuracy
y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0.6, 1, 0)

189/189 [==============================] - 2s 6ms/step


In [32]:
#@ Confusion Matrix
confusion_matrix(y_test, y_pred)

array([[3168,  251],
       [ 295, 2321]])

In [33]:
#@ Calculating accuracy score
print('The accuracy score is:',accuracy_score(y_test, y_pred))

The accuracy score is: 0.9095277547638774


In [34]:
#@ Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92      3419
           1       0.90      0.89      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



**The END**